## DAE - Denoising Auto-Encoder

In this auto-encoder, we corrupt the input by slightly adding noise to it and train the network to reconstruct the original input. This can be achieved in multiple ways. 
1. Add dropout to the input. This will randomly turn off few inputs, which acts as noise. (We'll use this)
2. Add gaussian or uniform noise to the input

We'll see how classification performance varies with respect to RAW Vs. Encoded features

In [2]:
# import tensorflow as tf
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# session = tf.Session(config=config, ...)

In [2]:
# Import the necessary libraries/modules
import numpy as np # for array operations
from keras.models import Model, Sequential # for defining the architectures
from keras.layers import Dense, Dropout, Input # layers for building the network
from keras.utils import to_categorical # to_categorical does one-hot encoding

# We'll use only 10,000 out of 50,000 samples for this 
tmp = np.load('cifar_pca_train.npz') # '.npz' is a dictionary which can hold many arrays
train_data = tmp['data'][:10000]     # 'data' holds the train data
train_labels = tmp['labels'][:10000] # 'labels' hold the corresponding labels for the above data

tmp = np.load('cifar_pca_test.npz')
test_data = tmp['data']
test_labels = tmp['labels']


# Converting labels into one-hot vectors for training. one-hot encoding is nothing but dummyfing
train_labels = to_categorical(train_labels, 10) 
test_labels = to_categorical(test_labels, 10)

print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)

Using TensorFlow backend.


(10000, 781)
(10000, 10)
(10000, 781)
(10000, 10)


### MLP for classifying cifar data using original features

In [2]:
# training a two hidden layer MLP for classification task

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(781,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))

mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

nb_epoch = 50      # number of epochs
batch_size = 32    # batch size
history = mlp.fit(train_data, train_labels,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    validation_data=(test_data, test_labels))

Train on 10000 samples, validate on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 4s - loss: 2.1546 - acc: 0.2270 - val_loss: 1.8624 - val_acc: 0.3383
Epoch 2/50
10000/10000 [==============================] - 1s - loss: 1.9159 - acc: 0.3061 - val_loss: 1.8166 - val_acc: 0.3701
Epoch 3/50
10000/10000 [==============================] - 1s - loss: 1.8421 - acc: 0.3433 - val_loss: 1.7608 - val_acc: 0.3819
Epoch 4/50
10000/10000 [==============================] - 1s - loss: 1.7798 - acc: 0.3691 - val_loss: 1.7697 - val_acc: 0.3660
Epoch 5/50
10000/10000 [==============================] - 2s - loss: 1.7501 - acc: 0.3827 - val_loss: 1.7544 - val_acc: 0.3734
Epoch 6/50
10000/10000 [==============================] - 1s - loss: 1.7107 - acc: 0.3990 - val_loss: 1.7267 - val_acc: 0.3867
Epoch 7/50
10000/10000 [==============================] - 1s - loss: 1.6862 - acc: 0.4063 - val_loss: 1.7223 - val_acc: 0.3914
Epoch 8/50
10000/10000 [==============================] - 1s 

### Training an Auto-Encoder

In [3]:
# Building an auto-encoder architecture with 'Model' function. This is a bit defferent from 'Sequential' type.
# For this, we need to create a series of layers connected together. 
# Once we have the connections in place, we can use model to define the architecture.
# To 'Model', we simply mention the first layer and the last layer.

nb_epoch = 50      # number of epochs
batch_size = 32    # batch size

input_img = Input(shape=(781,))
crrpt_img = Dropout(0.5)(input_img)
encoded = Dense(1000, activation='sigmoid')(crrpt_img)
decoded = Dense(781, activation='linear')(encoded)

autoencoder = Model(input_img,decoded)
autoencoder.compile(optimizer='adam',
                    loss='mean_squared_error')

history = autoencoder.fit(train_data, train_data,  
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_data, test_data))

autoencoder.save('DAE_l1_model.h5') # save the model weights

Train on 10000 samples, validate on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 1s - loss: 0.6812 - mean_squared_error: 0.6812 - val_loss: 0.3500 - val_mean_squared_error: 0.3500
Epoch 2/50
10000/10000 [==============================] - 1s - loss: 0.5708 - mean_squared_error: 0.5708 - val_loss: 0.2852 - val_mean_squared_error: 0.2852
Epoch 3/50
10000/10000 [==============================] - 1s - loss: 0.5476 - mean_squared_error: 0.5476 - val_loss: 0.2601 - val_mean_squared_error: 0.2601
Epoch 4/50
10000/10000 [==============================] - 1s - loss: 0.5385 - mean_squared_error: 0.5385 - val_loss: 0.2485 - val_mean_squared_error: 0.2485
Epoch 5/50
10000/10000 [==============================] - 1s - loss: 0.5316 - mean_squared_error: 0.5316 - val_loss: 0.2373 - val_mean_squared_error: 0.2373
Epoch 6/50
10000/10000 [==============================] - 1s - loss: 0.5302 - mean_squared_error: 0.5302 - val_loss: 0.2328 - val_mean_squared_error: 0.2328
Epoch 7/

In [4]:
# For predicting encoded output, we can define a model which starts with input layer and ends with encoded layer
# Since these layers are already trained, we can directly predict the encoded values

encoder = Model(input_img,encoded)
htrain_data = encoder.predict(train_data)

### MLP for classifying cifar data using encoded features

In [4]:
from keras.optimizers import Adam
adam = Adam(lr=0.001)

In [5]:
# training an MLP on autoencoder features

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(1000,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))

mlp.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

htest_data = encoder.predict(test_data)
print(htest_data.shape, test_labels.shape)
history = mlp.fit(htrain_data[:10000], train_labels[:10000],
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    validation_data=(htest_data, test_labels))

((10000, 1000), (10000, 10))
Train on 10000 samples, validate on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 1s - loss: 2.0458 - acc: 0.2645 - val_loss: 1.8417 - val_acc: 0.3386
Epoch 2/50
10000/10000 [==============================] - 1s - loss: 1.8226 - acc: 0.3504 - val_loss: 1.7803 - val_acc: 0.3662
Epoch 3/50
10000/10000 [==============================] - 1s - loss: 1.7520 - acc: 0.3779 - val_loss: 1.7306 - val_acc: 0.3822
Epoch 4/50
10000/10000 [==============================] - 1s - loss: 1.7034 - acc: 0.3979 - val_loss: 1.7332 - val_acc: 0.3840
Epoch 5/50
10000/10000 [==============================] - 1s - loss: 1.6570 - acc: 0.4159 - val_loss: 1.7385 - val_acc: 0.3847
Epoch 6/50
10000/10000 [==============================] - 1s - loss: 1.6137 - acc: 0.4310 - val_loss: 1.6896 - val_acc: 0.3938
Epoch 7/50
10000/10000 [==============================] - 1s - loss: 1.5799 - acc: 0.4465 - val_loss: 1.6942 - val_acc: 0.3964
Epoch 8/50
10000/10000 [========